<a href="https://colab.research.google.com/github/ReAlex1902/Oil_production/blob/master/scripts/Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Newral Networks

###### Importing libraries and preprocessing the data

In [1]:
import torch
import torch.nn as nn

import seaborn as sns
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# PATH = os.path.join('..', 'data', 'production.csv')
PATH = os.path.join('production.csv')
data = pd.read_csv(PATH)
data.head()

FileNotFoundError: ignored

###### Plot some series

In [ ]:
APIs = data['API'].unique()
plt.title('Production series')
plt.ylabel('Production')
plt.xlabel('Months')
plt.grid(True)
plt.autoscale(axis='x',tight=True)
plt.plot(data[data['API'] == APIs[0]]['Liquid']); 

In [ ]:
dict_ = {}
for api in APIs:
    dict_[api] = list(data[data['API'] == api]['Liquid'])

df = pd.DataFrame(dict_)
df.head()

# Preprocess data and create train data

In [ ]:
train_window = 1

def create_inout_sequences(input_data, tw = 12):
    '''
    Divides the data to  those done befor train window and after
    in: input_data - input data (list)
        tw - train_window (int)
    out: tuple - 1st elem contains 12 records, 2nd contains 1 record
    '''
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [ ]:
from sklearn.preprocessing import MinMaxScaler

train_size = 12     ## you can choose the training size

all = {}            ## all_data
train_data = {}     ## train data (depends on train size)
train_data_normalized = {}  ## normalize data with MinMaxScaler
train_inout_seq = {}        ## create sequence for training
scaler = {}                 ## scaler for each API

for i, api in enumerate(APIs):

    all[api] = df[api].values.astype(float)
    train_data[api] = all[api][:train_size]

    scaler[api] = MinMaxScaler(feature_range=(-1, 1))   ## scler for data

    train_data_normalized[api] = scaler[api].fit_transform(train_data[api].reshape(-1, 1))
    train_data_normalized[api] = torch.FloatTensor(train_data_normalized[api]).view(-1)
    train_inout_seq[api] = create_inout_sequences(train_data_normalized[api], train_window)

In [ ]:
pd.DataFrame(train_data_normalized)

In [ ]:
train_inout_seq[api]

# LSTM

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size = 1, hidden_layer_size = 100, output_size = 1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size, bidirectional = True)

        self.linear = nn.Linear(2*hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [ ]:
model = LSTM()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0007)

###### Training

In [ ]:
epochs = 150

model.train()   ## enable train mode
for api in APIs:    ## checking each API
    print("{} / {}".format(np.where(APIs == api)[0][0], len(APIs)))

    for i in range(epochs):
        for sequence, labels in train_inout_seq[api]:
            optimizer.zero_grad()
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                            torch.zeros(1, 1, model.hidden_layer_size))

            y_pred = model(sequence)

            single_loss = loss_function(y_pred, labels)
            single_loss.backward()
            optimizer.step()

        if i%25 == 1:
            print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')
    
    test_inputs[api] = train_data_normalized[api][-train_window:].tolist()

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

###### Evaluation

In [ ]:
fut_pred = 12
test_inputs = {}
for api in APIs:        ## creating test inputs
    test_inputs[api] = train_data_normalized[api][-train_window:].tolist()
print(test_inputs[5005072170100])

In [ ]:
model.eval()
seq = {}

## evaluating the model
for api in APIs:
    for i in range(fut_pred):
        seq[api] = torch.FloatTensor(test_inputs[api][-train_window:])
        with torch.no_grad():
            model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                            torch.zeros(1, 1, model.hidden_layer_size))
            test_inputs[api].append(model(seq[api]).item())

In [ ]:
test_inputs[5005072170100]

In [ ]:
actual_predictions = {}

for api in APIs:
    actual_predictions[api] = scaler[api].inverse_transform(np.array(test_inputs[api][train_window:] ).reshape(-1, 1))
print(actual_predictions[api])

# for api in APIs:
#     print(test_inputs[api][train_window:])

In [ ]:
x = np.arange(12, 24, 1)
print(x)

In [ ]:
for api in APIs:

    plt.title(f'API == {api}')
    plt.ylabel('Total Passengers')
    plt.grid(True)
    plt.autoscale(axis='x', tight=True)
    plt.plot(df[api][:12], label = 'Actual')
    plt.plot(x,actual_predictions[api], label = 'Prediction')
    plt.legend()
    plt.show()